In [16]:
import pandas as pd
import numpy as np
from IPython.display import display, Markdown
import pulp

model = pulp.LpProblem("Maximising profits for Tetravaal", pulp.LpMinimize)

variable_names = ['X1', 'X2', 'X3', 'X4']

variables = pulp.LpVariable.dicts("grams of ingredient",
                                     (i for i in variable_names),
                                     lowBound=0,
                                     cat='Continuous')

coefficients = [2.5, 2, 3, 2.24]

# Objective function
model += (
    pulp.lpSum([
        coefficients[i] * variables[variable_names[i]]
        for i in range(len(variable_names))])
), "Cost"


# And the constraints, the Matrix A
A=[[1, 0, 0, 0], #Coefficients of the first constraint
   [0, 1, 0, 0], #Coefficients of the second constraint
   [0, 0, 1, 0], #Coefficients of the third constraint
   [0, 0, 0, 1]] #Coefficients of the fourth constraint


# And vector b
b = [2000, 1900, 1000, 1200] #limits of the availability constraints

constraint_names = ['Eggplant', 'Zuccini', 'Pepper', 'Tomatoes']

# Constraints
for i in range(len(A)):
    model += pulp.lpSum([
        A[i][j] * variables[variable_names[j]]
        for j in range(len(variable_names))]) <= b[i] , constraint_names[i]

A2 = [[1, 1, 1, 1],
      [1, 1, -1, -1]]

b2 = [2000, 0]

constraint_names2 = ['Total weight', 'Quality']

for i in range(len(A2)):
    model += pulp.lpSum([
        A2[i][j] * variables[variable_names[j]]
        for j in range(len(variable_names))]) >= b2[i] , constraint_names2[i]

    # Solve our problem
model.solve(solver=pulp.GUROBI(msg = 0))
print(pulp.LpStatus[model.status])

Optimal


C:\Users\franc\PycharmProjects\operations-research-notebooks\venv\Lib\site-packages\pulp\pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [17]:
total_profit = pulp.value(model.objective)
display(Markdown("Total profit is %0.2f €"%total_profit))

display(Markdown("The following table shows the decision variables: "))
var_df = pd.DataFrame.from_dict(variables, orient="index",
                                columns = ["Variables"])

var_df["Solution (GRB)"] = var_df["Variables"].apply(lambda item: "{:.2f}".format(item.solverVar.X))
var_df["Reduced cost (GRB)"] = var_df["Variables"].apply(lambda item: "{:.2f}".format(item.solverVar.RC))
var_df["Objective Coefficient (GRB)"] = var_df["Variables"].apply(lambda item: "{:.2f}".format(item.solverVar.Obj))
var_df["Objective Lower bound (GRB)"] = var_df["Variables"].apply(lambda item: "{:.2f}".format(item.solverVar.SAObjLow) if item.solverVar.SAObjLow > -0.1 else "-Inf" )
var_df["Objective Upper bound (GRB)"] = var_df["Variables"].apply(lambda item: "{:.2f}".format(item.solverVar.SAObjUp) if item.solverVar.SAObjUp != item.solverVar.UB else "Inf")


display(var_df)


const_dict = dict(model.constraints)
con_df = pd.DataFrame.from_records(list(const_dict.items()), exclude=["Expression"], columns=["Constraint", "Expression"])
con_df["Right Hand Side"]=con_df["Constraint"].apply(lambda item: "{:.2f}".format(const_dict[item].solverConstraint.RHS))
con_df["Shadow Price"]=con_df["Constraint"].apply(lambda item: "{:.2f}".format(const_dict[item].solverConstraint.Pi))
con_df["Slack"]=con_df["Constraint"].apply(lambda item: "{:.2f}".format(const_dict[item].solverConstraint.Slack))
con_df["Min RHS"]=con_df["Constraint"].apply(lambda item: "{:.2f}".format(const_dict[item].solverConstraint.SARHSLow) )
con_df["Max RHS"]=con_df["Constraint"].apply(lambda item: "{:.2f}".format(const_dict[item].solverConstraint.SARHSUp) if const_dict[item].solverConstraint.SARHSUp < 1e10 else "Inf" )


print("The following table shows the constraints: ")
display(con_df)

Total profit is 4024.00 €

The following table shows the decision variables: 

,Variables,Solution (GRB),Reduced cost (GRB),Objective Coefficient (GRB),Objective Lower bound (GRB),Objective Upper bound (GRB)
X1,grams_of_ingredient_X1,0.00,0.26,2.50,2.24,Inf
X2,grams_of_ingredient_X2,1900.00,0.00,2.00,-Inf,2.24
X3,grams_of_ingredient_X3,0.00,0.76,3.00,2.24,Inf
X4,grams_of_ingredient_X4,100.00,0.00,2.24,2.00,2.50


The following table shows the constraints: 


,Constraint,Right Hand Side,Shadow Price,Slack,Min RHS,Max RHS
0,Eggplant,2000.00,0.00,2000.00,0.00,Inf
1,Zuccini,1900.00,-0.24,0.00,1000.00,2000.00
2,Pepper,1000.00,0.00,1000.00,0.00,Inf
3,Tomatoes,1200.00,0.00,1100.00,100.00,Inf
4,Total_weight,2000.00,2.24,0.00,1900.00,3100.00
5,Quality,0.00,0.00,-1800.00,-inf,1800.00
